In [ ]:
Working with QDA - a nonlinear LDA

In [4]:
from pandas_datareader import data

%matplotlib inline

from pandas_datareader import data
import pandas as pd

tickers = ["F", "TM", "GM", "TSLA"]

first_date = '2009-01-01'
last_date = '2016-12-31'


stock_panel = data.DataReader(tickers, 'google', first_date, last_date)

stock_df = stock_panel.Close.dropna()

classes = (stock_df.shift(-180) > stock_df).astype(int)

X = stock_panel.to_frame()
classes = classes.unstack()
classes = classes.swaplevel(0, 1).sort_index()
classes = classes.to_frame()
classes.index.names = ['Date', 'minor']
data = X.join(classes).dropna()
data.rename(columns={0: 'is_higher'}, inplace=True)

import patsy
X = patsy.dmatrix("Open + High + Low + Close + Volume + is_higher - 1", data.reset_index(),return_type='dataframe')

In [5]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
qda = QDA()

qda.fit(X.iloc[:, :-1], X.iloc[:, -1])
predictions = qda.predict(X.iloc[:, :-1])
predictions.sum()

23.0

In [6]:
from sklearn.metrics import classification_report
print classification_report(X.iloc[:, -1].values, predictions)

             precision    recall  f1-score   support

        0.0       0.86      0.99      0.92       845
        1.0       0.70      0.10      0.18       156

avg / total       0.83      0.85      0.80      1001



In [8]:
from sklearn.model_selection import ShuffleSplit
import scipy.stats as sp

shuffle_split_inst = ShuffleSplit()

for test, train in shuffle_split_inst.split(X):
      train_set = X.iloc[train]
      train_close = train_set.Close

      train_0 = train_close[~train_set.is_higher.astype(bool)]
      train_1 = train_close[train_set.is_higher.astype(bool)]

      test_set = X.iloc[test]
      test_close = test_set.Close.values

ll_0 = sp.norm.pdf(test_close, train_0.mean())
ll_1 = sp.norm.pdf(test_close, train_1.mean())

(ll_0 > ll_1).mean()

0.25444444444444442